In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def radial_frequency_filter(image, filter_type='bandreject', radius=50, bandwidth=10):
    """
    Apply a radial frequency filter (bandreject or bandpass) to an image.

    Parameters:
        image (ndarray): Input 2D grayscale image
        filter_type (str): 'bandreject' or 'bandpass'
        radius (float): Center frequency of the radial filter
        bandwidth (float): Width of the band

    Returns:
        filtered_image (ndarray): Image filtered in frequency domain
    """
    # 1. FFT and shift
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)

    # 2. Construct radial mask
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2

    Y, X = np.ogrid[:rows, :cols]
    D = np.sqrt((X - ccol)**2 + (Y - crow)**2)

    if filter_type == 'bandreject':
        mask = np.logical_or(D < (radius - bandwidth/2), D > (radius + bandwidth/2)).astype(np.float32)
    elif filter_type == 'bandpass':
        mask = np.logical_and(D >= (radius - bandwidth/2), D <= (radius + bandwidth/2)).astype(np.float32)
    else:
        raise ValueError("filter_type must be 'bandreject' or 'bandpass'")

    # 3. Apply filter in frequency domain
    f_filtered = fshift * mask

    # 4. Inverse FFT
    f_ishift = np.fft.ifftshift(f_filtered)
    img_filtered = np.fft.ifft2(f_ishift)
    img_filtered = np.abs(img_filtered)

    return img_filtered, mask

# Load grayscale image
image = np.genfromtxt('thinfilm-on-substrate.csv', delimiter=',')

# Apply radial bandreject filter
filtered_image, mask = radial_frequency_filter(image, filter_type='bandreject', radius=60, bandwidth=10)

# Display
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(image, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Radial Mask (Freq Domain)")
plt.imshow(mask, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Filtered Image")
plt.imshow(filtered_image, cmap='gray')
plt.axis('off')

plt.tight_layout()
plt.show()
